In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
working_dir = os.getcwd()
print(working_dir)

# get the parent directory
parent_dir = os.path.dirname(working_dir)
print(parent_dir)

# get the data directory
data_dir = os.path.join(parent_dir, 'data/5epochs')
print(data_dir)

/Users/dtjgp/Learning/Thesis/GreenAI/M1chip/Third_part/FL_code
/Users/dtjgp/Learning/Thesis/GreenAI/M1chip/Third_part
/Users/dtjgp/Learning/Thesis/GreenAI/M1chip/Third_part/data/5epochs


In [3]:
# find all the folders in the data directory
folders = os.listdir(data_dir)
print(folders)

model_path = []
# open each folder and get the .pth files
for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    # print(folder_path)
    files = os.listdir(folder_path)
    # print(files)
    for file in files:
        if file.endswith('net_state_dict_alexnet.pth'):
            file_path = os.path.join(folder_path, file)
            # print(file_path)
            # print('*************')
            model_path.append(file_path)

print(model_path)
            



['net2', 'net1']
['/Users/dtjgp/Learning/Thesis/GreenAI/M1chip/Third_part/data/5epochs/net2/net_state_dict_alexnet.pth', '/Users/dtjgp/Learning/Thesis/GreenAI/M1chip/Third_part/data/5epochs/net1/net_state_dict_alexnet.pth']


In [4]:
import torch
from torch import nn
from d2l import torch as d2l

In [5]:
# Define your model architecture as provided
net = nn.Sequential(
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

model_paths = model_path

# Load all model state dictionaries
state_dicts = [torch.load(path, map_location='cpu') for path in model_paths]

# Calculate the average of the parameters
avg_state_dict = {}
for key in state_dicts[0]:
    # Sum all parameters
    sum_param = sum(state_dict[key] for state_dict in state_dicts)
    # Take average
    avg_state_dict[key] = sum_param / len(state_dicts)

# Update the combined model with the averaged parameters
net.load_state_dict(avg_state_dict)

# Save the combined model
torch.save(net.state_dict(), os.path.join(data_dir, 'combined_model_state_dict.pth'))

In [6]:
# load the combined model
net.load_state_dict(torch.load(os.path.join(data_dir, 'combined_model_state_dict.pth')))

# load the test dataset
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

# set the model to evaluation mode
test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
print(test_acc)

0.1168
